# xAPI-SG Processor

### Processor of traces following the Experience API for Serious Games Profile (xAPI-SG)

**xAPI-SG main reference:**
*Applying standards to systematize learning analytics in serious games.
Ángel Serrano-Laguna, Iván Martínez-Ortiz, Jason Haag, Damon Regan, Andy Johnson, Baltasar Fernández-Manjón
Computer Standards & Interfaces 50 (2017) 116–123, http://dx.doi.org/10.1016/j.csi.2016.09.014*

Further info on GitHub wiki page: https://github.com/e-ucm/rage-analytics/wiki/xAPI-SG-Profile

The following code:
- expects as input a JSON file with a list of xAPI-SG statements
- analyzes the xAPI-SG statements and fills an adaptation of the default set of visualizations
https://github.com/e-ucm/rage-analytics/wiki/Default-visualizations-teacher

In [ ]:
local = True #Set to False if working with a web-hosted Jupyter server
import json
import numpy as np
from datetime import datetime, timedelta
import copy
import math
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
from ipywidgets import interact, interactive, fixed, HBox, Layout,VBox
import ipywidgets as widgets
from IPython.display import display, clear_output
import pprint
plt.style.use('dark_background')
plt.rcParams['figure.figsize']=[10, 8]

***Display all* function** is constructing an app to display all vis

In [ ]:
players_info = {} # dict with players info
timeformats=['%Y-%m-%dT%H:%M:%SZ','%Y-%m-%dT%H:%M:%S.%fZ'] #array of time format

%run fileBrowserAndUploadButtonToLoadProcessStatements.ipynb
%run globalsSelectors.ipynb

tabsTitle=["Start/Complete games","Videos","Progress","Completable","Alternative","Items interactions","Accessibles","Menus"]

def displayAllVisualisations():
    #Pie chart with number of games started and completed
    %run vis/xAPISG-GamesStartedCompleted.ipynb
    games_started_completed=vis_games_started_completed()
    tab_game_started_completed=VBox([
        widgets.HTML("<h3>Use the playerSelected widget to select the players you want to analyse</h3>"),
        checkboxesPlayersSelected.multi_select,
        games_started_completed
    ])
    #Bar chart with number of times each video was seen and skipped
    %run vis/xAPISG-VideosSeenSkipped.ipynb
    videoseenskipped=vis_videos_seen_skipped()
    tabvideoseen=VBox([
        widgets.HTML("<h3>Use the selectors widget to select the players and videos you want to analyse</h3>"),
        HBox([
            checkboxesPlayersSelected.multi_select,
            checkboxesVideoSelected.multi_select
        ]),
        videoseenskipped
    ])
    #Line chart with progress of players per time
    %run vis/xAPISG-PlayersProgress.ipynb
    playerprogress=vis_players_progress()
    tabplayerprogress=VBox([
        widgets.HTML("<h3>Use the playerSelected widget to select the players you want to analyse</h3>"),
        checkboxesPlayersSelected.multi_select,
        playerprogress
    ])
    #Bar chart with scores of players in each completable
    %run vis/xAPISG-CompletablesScores.ipynb
    completable_score=vis_completables_scores()
    #Bar chart with progress of players in each completable
    %run vis/xAPISG-CompletablesProgress.ipynb
    completable_progress=vis_completables_progress()
    #Bar chart with max and min completion time in each completable
    %run vis/xAPISG-CompletablesTimes.ipynb
    completable_time=vis_completables_times()
    completabletab=VBox([
        widgets.HTML("<h3>Use the Selectors widgets to select the players and completables you want to analyse</h3>"),
        HBox([
            checkboxesPlayersSelected.multi_select,
            checkboxesCompletableSelected.multi_select
        ]),
        completable_score,
        completable_progress,
        completable_time
    ])
    #Bar chart with number of correct and incorrect answers per player
    %run vis/xAPISG-CorrectIncorrectPerPlayer.ipynb
    #Bar chart with number of correct and incorrect answers per question (alternative)
    %run vis/xAPISG-CorrectIncorrectPerQuestion.ipynb
    #Bar chart per question, with number of times each alternative has been selected
    %run vis/xAPISG-AlternativesSelectedQuestion.ipynb
    alternativetab=VBox([
        widgets.HTML("<h3>Use the Selectors widgets to select the players and alternatives you want to analyse</h3>"),
        HBox([
            checkboxesPlayersSelected.multi_select,
            checkboxesAlternativeSelected.multi_select
        ]),
        vis_correct_incorrect_player(),
        vis_correct_incorrect_question(),
        vis_alternatives_selected_question()
    ])
    #item interacted
    %run vis/xAPISG-ItemsInteracted.ipynb
    %run vis/xAPISG-ItemsActionTypeInteracted.ipynb
    tabItem=VBox([
        widgets.HTML("<h3>Use the Selectors widgets to select the players and items you want to analyse</h3>"),
        HBox([
            checkboxesPlayersSelected.multi_select,
            checkboxesItemInteractedSelected.multi_select
        ]),
        vis_items_interacted_times_heatmap_interacted(),
        vis_items_interacted_times_interact(),
        vis_items_action_type_interacted_times_interact()
    ])
    %run vis/xAPISG-AccessedAccessible.ipynb
    tabAccessible=VBox([
        widgets.HTML("<h3>Use the Selectors widgets to select the players and accessed zone you want to analyse</h3>"),
        HBox([
            checkboxesPlayersSelected.multi_select,
            checkboxesAccessibleSelected.multi_select
        ]),
        vis_accessibles_times_heatMap_interacted(),
        vis_accessibles_times_interacted()
    ])
    #menus selected
    %run vis/xAPISG-MenusSelected.ipynb
    menusTab=VBox([
        widgets.HTML("<h3>Use the Selectors widgets to select the players and menus you want to analyse</h3>"),
        HBox([
            checkboxesPlayersSelected.multi_select,
            checkboxesSelectedmenus.multi_select
        ]),
        vis_menu_selected_interactive()
    ])
    global tabs
    tabs=widgets.Tab(children=[
        tab_game_started_completed,
        tabvideoseen,
        tabplayerprogress,
        completabletab,
        alternativetab,
        tabItem,
        tabAccessible,
        menusTab
    ])
    for i in range(len(tabsTitle)):
        tabs.set_title(i,tabsTitle[i])
    tabs.selected_index=previoustab.selected_index
    widgets.link((tabs,'selected_index'),(previoustab,'selected_index'))
    display(tabs)

header=widgets.HTML("<h1>xAPI-SG Processor</h1> <h2>Please select .json xAPI SG file to process this file and see visualisations</h2>")
footer=widgets.HTML("<h4>xAPI-SG Processor, by eUCM research team</h4>")
if local:
    app=VBox([header,fileBrowser.widget(),footer])
    display(app)
else:
    app=VBox([header,uploadButtonApp,footer])
    display(app)
    with outTabs:
        display_checkboxes()
        clear_output(wait=True)
    displayvis(None)

